# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import googlemaps

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_file = pd.read_csv("../output_data/city.csv")
weather_file


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Hermanus,0,ZA,1619826442,80,-34.4187,19.2345,53.01,6.93
1,1,Rikitea,90,PF,1619826442,78,-23.1203,-134.9692,78.93,20.69
2,2,Saint-Philippe,0,RE,1619826443,78,-21.3585,55.7679,75.20,6.91
3,3,Petropavlivka,99,UA,1619826443,81,48.4564,36.4367,56.34,7.54
4,4,Bluff,95,NZ,1619826443,81,-46.6000,168.3333,52.00,5.01
5,5,Yenangyaung,0,MM,1619826443,65,20.4667,94.8833,82.04,4.85
6,6,Oddur,90,SO,1619826446,81,4.1213,43.8894,74.14,8.10
7,7,Jamestown,90,US,1619826205,65,42.0970,-79.2353,42.01,11.50
8,8,Punta Arenas,90,CL,1619826447,93,-53.1500,-70.9167,42.80,3.44
9,9,Barrow,90,US,1619826447,79,71.2906,-156.7887,21.20,4.61


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_file[["Lat", "Lng"]]
humidity = weather_file["Humidity"].astype(float)

In [6]:
hfigure = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
hfigure.add_layer(heat_layer)
#Display figure
hfigure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
possible_vacay = weather_file.loc[(weather_file["Wind Speed"] <= 10) & (weather_file["Cloudiness"] == 0) & \
                                   (weather_file["Max Temp"] >= 70) & (weather_file["Max Temp"] <= 80)].dropna()

possible_vacay

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
2,2,Saint-Philippe,0,RE,1619826443,78,-21.3585,55.7679,75.2,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
possible_hotels = possible_vacay.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
possible_hotels["Hotel Name"] = ""
possible_hotels

,City,Country,Lat,Lng,Hotel Name
2,Saint-Philippe,RE,-21.3585,55.7679,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
for index, row in possible_hotels.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    #base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    json.dumps(hotel_name, indent=4, sort_keys=True)
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        possible_hotels.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
possible_hotels

,City,Country,Lat,Lng,Hotel Name
2,Saint-Philippe,RE,-21.3585,55.7679,Les Embruns Du Baril


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in possible_hotels.iterrows()]
locations = possible_hotels[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

hfigure.add_layer(markers)

# Display Map
hfigure


Figure(layout=FigureLayout(height='420px'))